In [1]:
# Load all helper function
%run -i 'source.py'

# Preparation

# Feature Engineering

In [13]:
train = pd.read_feather("../processing/train_raw.feather")

- We only use 1 day! But it's enough to generate insights!

## Time stamp

In [20]:
# Transformed it into local time in China. 
train['click_time'] = train['click_time'] + timedelta(hours=8)

In [21]:
# To Prototype we downsize the data
train = train[train['click_time'].dt.day == 8] 

In [22]:
# Add day, hour, and minute
dt = train['click_time'].dt
train['day_of_week'] = dt.dayofweek
train['day'] = dt.day.astype('uint8')
train['hour'] = dt.hour.astype('uint8')
train['minute'] = dt.minute.astype('uint8')

## Click amount and conversion rate to download

In [23]:
for group in tqdm([['ip'],
                   ['app'],
                   ['device'],
                   ['os'],
                   ['channel'],
                   ['hour'],
                   ['app', 'channel'],
                   ['ip', 'hour'],
                   ['ip', 'device', 'app'],
                   ['ip', 'device', 'os'],
                   ['ip', 'device', 'os', 'hour']]):
    feature_suffix = '_'.join(group)
    train['click_freq_by_' + feature_suffix] = train.groupby(group)['click_time'].transform('count').astype('int')
    # train['download_freq_by_' + feature_suffix] = train.groupby(group)['is_attributed'].transform('sum').astype('int') # download times by ip
    download_freq = train.groupby(group)['is_attributed'].transform('sum').astype('int') # download times by ip
    train['download_rate_by_' + feature_suffix] = download_freq / train['click_freq_by_' + feature_suffix]

100%|██████████| 11/11 [01:32<00:00,  8.43s/it]


In [24]:
train = reduce_mem_usage(train)

-------------------Begin downsizing--------------------
ip converted from int32 to int32
app converted from int16 to int16
device converted from int16 to int16
os converted from int16 to int16
channel converted from int16 to int16
day_of_week converted from int64 to int8
click_freq_by_ip converted from int64 to int32
download_rate_by_ip converted from float64 to float16
click_freq_by_app converted from int64 to int32
download_rate_by_app converted from float64 to float16
click_freq_by_device converted from int64 to int32
download_rate_by_device converted from float64 to float16
click_freq_by_os converted from int64 to int32
download_rate_by_os converted from float64 to float16
click_freq_by_channel converted from int64 to int32
download_rate_by_channel converted from float64 to float16
click_freq_by_hour converted from int64 to int32
download_rate_by_hour converted from float64 to float16
click_freq_by_app_channel converted from int64 to int32
download_rate_by_app_channel converted fro

## Unique values from group

In [25]:
# Looping features like train['nunique_channel_per_ip'] = train.groupby(['ip'])['channel'].transform('nunique')
for group, counted in tqdm([(['ip'], ['channel']),
                           (['ip'], ['device']),
                           (['ip'], ['app']),
                           (['ip'], ['os']),
                           (['app'], ['channel'])]):
    group_suffix = '_'.join(group)
    counted_suffix = '_'.join(counted)
    train['nunique_' + counted_suffix + '_per_' + group_suffix] = train.groupby(group_suffix)[counted_suffix].transform('nunique')

100%|██████████| 5/5 [01:53<00:00, 22.68s/it]


## Click Pattern

```python
# This feature takes some time to run, and need some missing value handling job! 

train.sort_values(by = "click_time", ascending=True, ignore_index=True, inplace=True)
train['gap_last_click'] = train['click_time'] - train.groupby(['ip', 'device', 'os'])['click_time'].transform(lambda x: x.shift(1))
train = train.dropna(subset=['gap_last_click']).reset_index(drop=True)
train['gap_last_click'] = train['gap_last_click'].dt.seconds
```

- This `gap_last_click` feature is important!! But we can test how to handle it's missing for initial click, which leads to download sometimes for real user. We should discuss this when we are doing modeling part!

- Right now, we just move on

In [26]:
train = reduce_mem_usage(train)

-------------------Begin downsizing--------------------
ip converted from int32 to int32
app converted from int16 to int16
device converted from int16 to int16
os converted from int16 to int16
channel converted from int16 to int16
click_freq_by_ip converted from int32 to int32
download_rate_by_ip converted from float16 to float16
click_freq_by_app converted from int32 to int32
download_rate_by_app converted from float16 to float16
click_freq_by_device converted from int32 to int32
download_rate_by_device converted from float16 to float16
click_freq_by_os converted from int32 to int32
download_rate_by_os converted from float16 to float16
click_freq_by_channel converted from int32 to int32
download_rate_by_channel converted from float16 to float16
click_freq_by_hour converted from int32 to int32
download_rate_by_hour converted from float16 to float16
click_freq_by_app_channel converted from int32 to int32
download_rate_by_app_channel converted from float16 to float16
click_freq_by_ip_hou

## Save the output

In [27]:
train = train.reset_index(drop=True)

In [28]:
train.to_feather("../processing/train_with_feature.feather")

# Generate Feature for unseen data

In [2]:
unseen_data = pd.read_feather("../processing/test_raw.feather")
unseen_data['click_time'] = unseen_data['click_time'] + timedelta(hours=8)
unseen_data['hour'] = unseen_data['click_time'].dt.hour.astype('uint8')

In [3]:
unseen_data

,click_id,ip,app,device,os,channel,click_time,hour
0,0,5744,9,1,3,107,2017-11-10 12:00:00,12
1,1,119901,9,1,3,466,2017-11-10 12:00:00,12
2,2,72287,21,1,19,128,2017-11-10 12:00:00,12
3,3,78477,15,1,13,111,2017-11-10 12:00:00,12
4,4,123080,12,1,13,328,2017-11-10 12:00:00,12
...,...,...,...,...,...,...,...,...
18790464,18790464,99442,9,1,13,127,2017-11-10 23:00:00,23
18790465,18790465,88046,23,1,37,153,2017-11-10 23:00:00,23
18790466,18790467,81398,18,1,17,265,2017-11-10 23:00:00,23
18790467,18790466,123236,27,1,13,122,2017-11-10 23:00:00,23


Nevigate the feature from the exsiting data usint 5 rules:

## Fully-map

> Use `["ip", "app", "device", "os", "channel","hour"]` to locate features in exsiting dataset.

In [4]:
dataset = pd.read_feather("../processing/train_with_feature.feather")

In [5]:
ids = ["ip","app","device","os","channel"]
time_feature = ["hour"]
numc_features = ['click_freq_by_ip',
                 'download_rate_by_ip',
                 'click_freq_by_app',
                 'download_rate_by_app',
                 'click_freq_by_device',
                 'download_rate_by_device',
                 'click_freq_by_os',
                 'download_rate_by_os',
                 'click_freq_by_channel',
                 'download_rate_by_channel',
                 'click_freq_by_hour',
                 'download_rate_by_hour',
                 'click_freq_by_app_channel',
                 'download_rate_by_app_channel',
                 'click_freq_by_ip_hour',
                 'download_rate_by_ip_hour',
                 'click_freq_by_ip_device_app',
                 'download_rate_by_ip_device_app',
                 'click_freq_by_ip_device_os',
                 'download_rate_by_ip_device_os',
                 'click_freq_by_ip_device_os_hour',
                 'download_rate_by_ip_device_os_hour',
                 'nunique_channel_per_ip',
                 'nunique_device_per_ip',
                 'nunique_app_per_ip',
                 'nunique_os_per_ip',
                 'nunique_channel_per_app']
dataset_feature = dataset[ids + time_feature + numc_features]

In [6]:
#dataset_feature_pure = dataset_feature.sample(n = 100000).drop_duplicates(subset=ids+time_feature, keep="first")
dataset_feature_pure = dataset_feature.drop_duplicates(subset=ids+time_feature, keep="first")

In [7]:
unseen_data_binded = unseen_data.merge(dataset_feature_pure, how = "left", indicator = True)

In [8]:
unseen_data_binded_exist_1 = unseen_data_binded.loc[unseen_data_binded["_merge"] == "both"].drop(labels='_merge', axis=1)
unseen_data_binded_exist_1

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
2,2,72287,21,1,19,128,2017-11-10 12:00:00,12,5103.00000,0.00118,...,0.00000,1490.00000,0.00201,104.00000,0.00000,114.00000,7.00000,52.00000,44.00000,8.00000
11,11,111025,3,1,19,137,2017-11-10 12:00:00,12,74629.00000,0.00042,...,0.00023,16636.00000,0.00024,854.00000,0.00000,140.00000,38.00000,113.00000,104.00000,45.00000
24,24,48240,18,1,42,107,2017-11-10 12:00:00,12,51025.00000,0.00033,...,0.00000,270.00000,0.00000,8.00000,0.00000,127.00000,21.00000,94.00000,121.00000,18.00000
27,27,114220,9,1,13,445,2017-11-10 12:00:00,12,50927.00000,0.00004,...,0.00000,10550.00000,0.00000,437.00000,0.00000,132.00000,38.00000,115.00000,96.00000,39.00000
39,39,5429,12,1,19,265,2017-11-10 12:00:00,12,649.00000,0.00308,...,0.00000,143.00000,0.00000,14.00000,0.00000,79.00000,6.00000,33.00000,35.00000,33.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18790448,18790448,25553,2,1,22,205,2017-11-10 23:00:00,23,14863.00000,0.00067,...,0.00000,789.00000,0.00380,47.00000,0.00000,108.00000,5.00000,49.00000,70.00000,26.00000
18790449,18790449,119531,21,2,19,128,2017-11-10 23:00:00,23,70473.00000,0.00121,...,0.00000,1746.00000,0.00057,35.00000,0.00000,133.00000,44.00000,115.00000,91.00000,8.00000
18790459,18790459,102467,15,1,17,140,2017-11-10 23:00:00,23,12967.00000,0.00224,...,0.00000,388.00000,0.00000,46.00000,0.00000,114.00000,9.00000,55.00000,68.00000,30.00000
18790465,18790465,88046,23,1,37,153,2017-11-10 23:00:00,23,3125.00000,0.00160,...,0.00000,170.00000,0.00000,28.00000,0.00000,107.00000,7.00000,52.00000,48.00000,8.00000


In [9]:
unseen_data_binded_new_1 = unseen_data_binded.loc[unseen_data_binded["_merge"] == "left_only"].drop(labels='_merge', axis=1)
unseen_data_binded_new_1

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
0,0,5744,9,1,3,107,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,119901,9,1,3,466,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,78477,15,1,13,111,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,123080,12,1,13,328,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,110769,18,1,13,107,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18790462,18790461,113418,17,1,17,128,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18790463,18790463,69245,12,1,13,135,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18790464,18790464,99442,9,1,13,127,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18790466,18790467,81398,18,1,17,265,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Locate feature without time

> Use `["ip", "app", "device", "os", "channel"]` to locate features in exsiting dataset.

In [10]:
temp = unseen_data_binded_new_1[["click_id","ip", "app", "device", "os", "channel","click_time", "hour"]].merge(dataset_feature_pure.drop(labels='hour', axis=1), 
                                                                                                                on=["ip", "app", "device", "os", "channel"], 
                                                                                                                how="left", 
                                                                                                                indicator = True)

In [11]:
unseen_data_binded_exist_2 = temp.loc[temp["_merge"] == "both"].drop(labels='_merge', axis=1)
unseen_data_binded_exist_2 = unseen_data_binded_exist_2.groupby(["click_id","ip", "app", "device", "os", "channel","hour"], as_index=False).mean()
unseen_data_binded_exist_2

,click_id,ip,app,device,os,channel,hour,click_freq_by_ip,download_rate_by_ip,click_freq_by_app,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
0,3,78477,15,1,13,111,12,4201.00000,0.00238,5603385.00000,...,0.00000,921.00000,0.00326,19.00000,0.00000,113.00000,5.00000,48.00000,43.00000,30.00000
1,5,110769,18,1,13,107,12,1346.00000,0.00074,5090081.00000,...,0.00000,276.00000,0.00000,20.00000,0.00000,102.00000,5.00000,38.00000,35.00000,18.00000
2,7,88637,27,1,19,153,12,1774.00000,0.00056,427276.00000,...,0.00000,393.00000,0.00000,18.00000,0.00000,93.00000,4.00000,40.00000,38.00000,5.00000
3,9,14932,18,1,10,107,12,2428.00000,0.00041,5090081.00000,...,0.00000,23.00000,0.00000,4.00000,0.00000,106.00000,3.00000,42.00000,39.00000,18.00000
4,12,16050,12,1,13,265,12,1709.00000,0.00059,8408754.00000,...,0.00000,191.00000,0.00000,9.50000,0.00000,99.00000,4.00000,41.00000,33.00000,33.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5673241,18790440,14334,14,1,19,401,23,1264.00000,0.00158,3347339.00000,...,0.00000,298.00000,0.00000,18.00000,0.00000,98.00000,4.00000,35.00000,36.00000,36.00000
5673242,18790447,66176,1,1,19,125,23,4971.00000,0.00040,1968988.00000,...,0.00000,1063.00000,0.00094,6.00000,0.00000,106.00000,5.00000,43.00000,67.00000,31.00000
5673243,18790456,100697,8,1,18,259,23,1038.00000,0.00193,946411.00000,...,0.00000,57.00000,0.00000,27.00000,0.00000,88.00000,5.00000,36.00000,44.00000,3.00000
5673244,18790458,85329,2,1,14,452,23,23839.00000,0.00180,7366228.00000,...,0.00053,242.00000,0.00000,24.00000,0.00000,130.00000,39.00000,102.00000,89.00000,26.00000


In [12]:
unseen_data_binded_new_2 = temp.loc[temp["_merge"] == "left_only"].drop(labels='_merge', axis=1)
unseen_data_binded_new_2

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
0,0,5744,9,1,3,107,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,119901,9,1,3,466,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,123080,12,1,13,328,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6,12540,3,1,1,137,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,8,123701,12,1,53,424,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29191817,18790461,113418,17,1,17,128,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29191818,18790463,69245,12,1,13,135,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29191819,18790464,99442,9,1,13,127,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29191820,18790467,81398,18,1,17,265,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Use ip+device to match features


> Use `["ip", "device"]` to locate features in exsiting dataset.

In [13]:
temp = unseen_data_binded_new_2[["click_id","ip", "app", "device", "os", "channel","click_time", "hour"]].merge(dataset_feature_pure.groupby(["ip", "device"], as_index=False)[numc_features].mean(),
                                                                                                                how="left", 
                                                                                                                indicator = True)

In [14]:
unseen_data_binded_exist_3 = temp.loc[temp["_merge"] == "both"].drop(labels='_merge', axis=1)
unseen_data_binded_exist_3

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
0,0,5744,9,1,3,107,2017-11-10 12:00:00,12,339.00000,0.00295,...,0.00000,27.10545,0.00000,6.46182,0.00000,67.00000,4.00000,27.00000,31.00000,28.10182
1,1,119901,9,1,3,466,2017-11-10 12:00:00,12,7396.00000,0.00203,...,0.00218,752.53691,0.00155,45.42412,0.00169,118.00000,10.00000,63.00000,54.00000,28.12297
3,6,12540,3,1,1,137,2017-11-10 12:00:00,12,1339.00000,0.00224,...,0.00096,182.88558,0.00073,20.68365,0.00096,99.00000,5.00000,39.00000,31.00000,28.06538
4,8,123701,12,1,53,424,2017-11-10 12:00:00,12,10149.00000,0.00049,...,0.00024,926.52186,0.00017,49.31823,0.00014,120.00000,18.00000,52.00000,76.00000,27.82684
6,13,31555,9,1,13,244,2017-11-10 12:00:00,12,1842.00000,0.00054,...,0.00083,201.48093,0.00036,19.45108,0.00066,103.00000,7.00000,42.00000,37.00000,27.70978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10167099,18790451,48536,14,1,19,442,2017-11-10 23:00:00,23,522.00000,0.00192,...,0.00217,85.24000,0.00201,16.92571,0.00114,71.00000,3.00000,28.00000,24.00000,28.30000
10167102,18790455,55874,1,1,22,125,2017-11-10 23:00:00,23,3366.00000,0.00059,...,0.00072,432.59630,0.00076,36.38219,0.00092,104.00000,6.00000,46.00000,52.00000,28.44772
10167106,18790461,113418,17,1,17,128,2017-11-10 23:00:00,23,4180.00000,0.00072,...,0.00054,484.81758,0.00054,33.90879,0.00063,115.00000,6.00000,53.00000,46.00000,28.13243
10167107,18790463,69245,12,1,13,135,2017-11-10 23:00:00,23,1077.00000,0.00000,...,0.00000,170.61538,0.00000,19.34135,0.00000,88.00000,2.00000,32.00000,31.00000,28.18870


In [15]:
unseen_data_binded_new_3 = temp.loc[temp["_merge"] == "left_only"].drop(labels='_merge', axis=1)
unseen_data_binded_new_3

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
2,4,123080,12,1,13,328,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10,106056,26,1,19,477,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,16,104009,18,1,17,107,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,19,80209,27,1,13,122,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,37,104791,6,1,13,459,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10167103,18790454,93291,9,1,19,145,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10167104,18790457,93291,20,1,19,259,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10167105,18790460,80537,9,1,19,445,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10167108,18790464,99442,9,1,13,127,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Use Only ip to match feature

In [16]:
temp = unseen_data_binded_new_3[["click_id","ip", "app", "device", "os", "channel","click_time", "hour"]].merge(dataset_feature_pure.groupby(["ip"], as_index=False)[numc_features].mean(),
                                                                                                                how="left", 
                                                                                                                indicator = True)

In [17]:
unseen_data_binded_exist_4 = temp.loc[temp["_merge"] == "both"].drop(labels='_merge', axis=1)
unseen_data_binded_exist_4

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
77,398,19964,19,16,0,213,2017-11-10 12:00:00,12,1930.00000,0.00052,...,0.00069,201.52831,0.00050,19.99793,0.00059,97.00000,6.00000,39.00000,50.00000,28.02624
184,927,48533,215,3,33,347,2017-11-10 12:00:01,12,4895.00000,0.00102,...,0.00111,428.73205,0.00102,37.00339,0.00129,106.00000,11.00000,51.00000,57.00000,27.48137
200,990,19964,19,16,0,213,2017-11-10 12:00:01,12,1930.00000,0.00052,...,0.00069,201.52831,0.00050,19.99793,0.00059,97.00000,6.00000,39.00000,50.00000,28.02624
318,1533,32480,97,3,33,347,2017-11-10 12:00:02,12,1247.00000,0.00160,...,0.00206,78.99236,0.00148,8.41703,0.00111,88.00000,5.00000,33.00000,49.00000,29.05677
340,1628,63893,3,5,45,404,2017-11-10 12:00:02,12,1100.00000,0.00182,...,0.00233,115.36205,0.00204,17.43073,0.00233,93.00000,4.00000,37.00000,40.00000,28.32945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069585,18790191,17977,21,2,10,232,2017-11-10 23:00:00,23,934.00000,0.00107,...,0.00164,115.14098,0.00164,16.43934,0.00164,91.00000,4.00000,36.00000,35.00000,28.91803
4069601,18790280,17977,9,2,10,134,2017-11-10 23:00:00,23,934.00000,0.00107,...,0.00164,115.14098,0.00164,16.43934,0.00164,91.00000,4.00000,36.00000,35.00000,28.91803
4069602,18790283,17977,12,2,10,145,2017-11-10 23:00:00,23,934.00000,0.00107,...,0.00164,115.14098,0.00164,16.43934,0.00164,91.00000,4.00000,36.00000,35.00000,28.91803
4069616,18790330,832,19,204,38,213,2017-11-10 23:00:00,23,2201.00000,0.00182,...,0.00247,324.39975,0.00191,25.09932,0.00191,96.00000,5.00000,42.00000,35.00000,28.45157


In [18]:
unseen_data_binded_new_4 = temp.loc[temp["_merge"] == "left_only"].drop(labels='_merge', axis=1)
unseen_data_binded_new_4

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
0,4,123080,12,1,13,328,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,106056,26,1,19,477,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,104009,18,1,17,107,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19,80209,27,1,13,122,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37,104791,6,1,13,459,2017-11-10 12:00:00,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069649,18790454,93291,9,1,19,145,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4069650,18790457,93291,20,1,19,259,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4069651,18790460,80537,9,1,19,445,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4069652,18790464,99442,9,1,13,127,2017-11-10 23:00:00,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Provide an average result

In [19]:
avg_impute = pd.DataFrame({"mean":dataset_feature_pure[numc_features].mean()}).T
avg_impute = avg_impute.reset_index(drop=True)
avg_impute['join_id'] = 1
avg_impute

/root/miniconda3/envs/myconda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/root/miniconda3/envs/myconda/lib/python3.7/site-packages/numpy/core/_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,click_freq_by_ip,download_rate_by_ip,click_freq_by_app,download_rate_by_app,click_freq_by_device,download_rate_by_device,click_freq_by_os,download_rate_by_os,click_freq_by_channel,download_rate_by_channel,...,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app,join_id
0,8228.38139,NaN,5996919.70222,NaN,56467485.84662,NaN,6770754.33617,NaN,1447331.24365,NaN,...,520.19583,NaN,36.51880,NaN,98.54734,8.27912,45.63491,44.08737,27.86215,1


In [20]:
temp = unseen_data_binded_new_4[["click_id","ip", "app", "device", "os", "channel","click_time", "hour"]]
temp['join_id'] = 1
temp

,click_id,ip,app,device,os,channel,click_time,hour,join_id
0,4,123080,12,1,13,328,2017-11-10 12:00:00,12,1
1,10,106056,26,1,19,477,2017-11-10 12:00:00,12,1
2,16,104009,18,1,17,107,2017-11-10 12:00:00,12,1
3,19,80209,27,1,13,122,2017-11-10 12:00:00,12,1
4,37,104791,6,1,13,459,2017-11-10 12:00:00,12,1
...,...,...,...,...,...,...,...,...,...
4069649,18790454,93291,9,1,19,145,2017-11-10 23:00:00,23,1
4069650,18790457,93291,20,1,19,259,2017-11-10 23:00:00,23,1
4069651,18790460,80537,9,1,19,445,2017-11-10 23:00:00,23,1
4069652,18790464,99442,9,1,13,127,2017-11-10 23:00:00,23,1


In [21]:
unseen_data_newip = temp.merge(avg_impute, how="left").drop(labels='join_id', axis=1)

In [22]:
unseen_data_withfeature = pd.concat([unseen_data_binded_exist_1, unseen_data_binded_exist_2, unseen_data_binded_exist_3, unseen_data_binded_exist_4, unseen_data_newip], axis=0)

In [23]:
unseen_data_withfeature = unseen_data_withfeature.sort_values(by = "click_id").reset_index(drop=True)
unseen_data_withfeature

,click_id,ip,app,device,os,channel,click_time,hour,click_freq_by_ip,download_rate_by_ip,...,download_rate_by_ip_device_app,click_freq_by_ip_device_os,download_rate_by_ip_device_os,click_freq_by_ip_device_os_hour,download_rate_by_ip_device_os_hour,nunique_channel_per_ip,nunique_device_per_ip,nunique_app_per_ip,nunique_os_per_ip,nunique_channel_per_app
0,0,5744,9,1,3,107,2017-11-10 12:00:00,12,339.00000,0.00295,...,0.00000,27.10545,0.00000,6.46182,0.00000,67.00000,4.00000,27.00000,31.00000,28.10182
1,1,119901,9,1,3,466,2017-11-10 12:00:00,12,7396.00000,0.00203,...,0.00218,752.53691,0.00155,45.42412,0.00169,118.00000,10.00000,63.00000,54.00000,28.12297
2,2,72287,21,1,19,128,2017-11-10 12:00:00,12,5103.00000,0.00118,...,0.00000,1490.00000,0.00201,104.00000,0.00000,114.00000,7.00000,52.00000,44.00000,8.00000
3,3,78477,15,1,13,111,NaT,12,4201.00000,0.00238,...,0.00000,921.00000,0.00326,19.00000,0.00000,113.00000,5.00000,48.00000,43.00000,30.00000
4,4,123080,12,1,13,328,2017-11-10 12:00:00,12,8228.38139,NaN,...,NaN,520.19583,NaN,36.51880,NaN,98.54734,8.27912,45.63491,44.08737,27.86215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18790464,18790464,99442,9,1,13,127,2017-11-10 23:00:00,23,8228.38139,NaN,...,NaN,520.19583,NaN,36.51880,NaN,98.54734,8.27912,45.63491,44.08737,27.86215
18790465,18790465,88046,23,1,37,153,2017-11-10 23:00:00,23,3125.00000,0.00160,...,0.00000,170.00000,0.00000,28.00000,0.00000,107.00000,7.00000,52.00000,48.00000,8.00000
18790466,18790466,123236,27,1,13,122,2017-11-10 23:00:00,23,8228.38139,NaN,...,NaN,520.19583,NaN,36.51880,NaN,98.54734,8.27912,45.63491,44.08737,27.86215
18790467,18790467,81398,18,1,17,265,2017-11-10 23:00:00,23,3640.00000,0.00137,...,0.00143,525.93625,0.00112,35.69482,0.00118,110.00000,6.00000,45.00000,43.00000,28.08845


## Save result

In [24]:
pd.read_csv("../input/sample_submission_adtracking.csv")

,click_id,is_attributed
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
18790464,18790464,0
18790465,18790465,0
18790466,18790467,0
18790467,18790466,0


In [25]:
unseen_data_withfeature.to_feather("../processing/unseen_withfeature.feather")